In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local[*]") \
        .appName('test') \
        .getOrCreate()

25/03/07 00:00:02 WARN Utils: Your hostname, BTT-IPEKGULER resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/07 00:00:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/07 00:00:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.version

'3.5.5'

In [3]:
df = spark.read \
    .option("header", "true") \
    .parquet('yellow_tripdata_2024-10.parquet') \
    .repartition(4)

df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2024-10-07 16:40:43|  2024-10-07 18:10:56|              1|         14.8|        99|                 N|         127|         225|           1|       47.5|  0.0|    0.5|       0.

In [5]:
df.write.parquet('yellow/2024/10', mode="overwrite")

In [6]:
df.registerTempTable('trips_data')

spark.sql("""
SELECT COUNT(*) AS count
FROM trips_data
WHERE DATE_TRUNC('day', tpep_pickup_datetime) = '2024-10-15 00:00:00';
""").show()

/home/ipekguler/anaconda3/lib/python3.9/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


+------+
| count|
+------+
|128893|
+------+



In [7]:
spark.sql("""
SELECT DATEDIFF(hour, tpep_pickup_datetime, tpep_dropoff_datetime) AS duration
FROM trips_data
ORDER BY duration DESC
LIMIT 1;
""").show()

+--------+
|duration|
+--------+
|     162|
+--------+



In [8]:
zone_df = spark.read \
        .option("header", "true") \
        .csv('taxi_zone_lookup.csv')

zone_df.createOrReplaceTempView("zone_data")

zone_df.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [9]:
spark.sql("""
SELECT COUNT(*),
PULocationID, Zone
FROM trips_data
LEFT JOIN zone_data
ON PULocationID = LocationID
GROUP BY PULocationID, Zone
ORDER BY COUNT(*) ASC
LIMIT 1;
""").show()

+--------+------------+--------------------+
|count(1)|PULocationID|                Zone|
+--------+------------+--------------------+
|       1|         105|Governor's Island...|
+--------+------------+--------------------+

